In [22]:
from keras.models import model_from_json

# Đọc cấu trúc model từ tệp .json
with open("model/model.json", "r") as json_file:
    loaded_model_json = json_file.read()
loaded_model = model_from_json(loaded_model_json)

# Tải trọng số vào model đã load
loaded_model.load_weights("model/model.weights.h5")

# Compile model (cần thiết để tiếp tục sử dụng model)
loaded_model.compile(optimizer='adam', loss='mae', metrics=['mse'])

In [23]:
#Importing cv2
import cv2
import numpy as np

# Loading cascades
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
# eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

# Resize lại ảnh  
def resize(img, size = (128, 128)):
    new_width = size[0]
    new_height = size[1]
    img_resized = cv2.resize(src=img, dsize=(new_width, new_height))
    return img_resized

# Trả về vị trí khuôn mặt trong ảnh
def detect(img): 
    faces = face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=5) 
    return faces

In [24]:
age = 0

# Tách mặt, resize lại ảnh và dự đoán
def process_and_predict(im, change):
    global age
    
    cur_im = np.copy(im)
    pos = detect(im)
    if(len(pos)>0):
        x1, y1, x2, y2 = pos[0]
        im = im[y1:y1+y2, x1:x1+x2, :]
        cur_im = cv2.rectangle(cur_im, (x1, y1), (x1+x2, y1+y2), (255,0,0), 3)
        im = resize(im)
        if change:
            ar = im
            ar = ar.astype('float32')
            ar /= 255.0
            ar = ar.reshape(1, 128, 128, 3)
            age = loaded_model.predict(ar, verbose=0)
        cur_im = cv2.putText(cur_im, 'Age:%d'%age, (x1, y1-10), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0))
    
    return cur_im


In [25]:
cam = cv2.VideoCapture(0)
c =False
while True:
    ret, image =cam.read()
    key = cv2.waitKey(1)
    if key == ord('p'):
        c = True
    if key == ord('q'):
        break
    if ret:
        cv2.imshow('Cam 1', process_and_predict(image, c))
    c = False
    
cam.release()
cv2.destroyAllWindows()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_12424\94246835.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cur_im = cv2.putText(cur_im, 'Age:%d'%age, (x1, y1-10), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0))
